# March Meeting 2022

In [23]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
from tenacity import retry, stop_after_attempt
from tenacity import wait_fixed, wait_exponential

In [37]:
@retry(stop = stop_after_attempt(10),
       wait = wait_exponential(multiplier=1, min=4, max=100))
def parse_session(sesssion_url):
    req = requests.get(sesssion_url)
    soup = BeautifulSoup(req.text, 'lxml')
    session_id = sesssion_url.split('/')[-1]
    session_links = []
    for link in soup.find_all('a'):
        link_url = str(link.get('href'))
        if session_id+'.' in link_url:
            full_link = "%s%s" % (radix, link_url[1:])
            session_links.append(full_link)
    this_session = {}
    this_session['id'] = session_id
    this_session['contents'] = []
    this_session['html'] = req.text
    if '403 Forbidden' in req.text:
        print("Failed...")
        raise Exception
    print(session_id, end=', ')
    for talk_url in session_links:
        this_talk = parse_talk(talk_url)
        this_session['contents'].append(this_talk)
        time.sleep(0.1)
    return this_session

@retry(stop = stop_after_attempt(10),
       wait = wait_exponential(multiplier=1, min=4, max=100))
def parse_talk(talk_url):
    this_talk = {}
    req = requests.get(talk_url)
    soup = BeautifulSoup(req.text, 'lxml')
    this_talk['html'] = req.text
    if '403 Forbidden' in this_talk['html']:
        print("Failed...")
        raise Exception
    this_talk['presenter'] = ''
    this_talk['abstract'] = ''
    for index, thing in enumerate(soup.find_all('h4')):
        if 'Presenter' in thing.text:
            this_talk['presenter'] = (thing.nextSibling.nextSibling.find_all('span')[1].text)
    # get the abstract of the presentation
    for stuff in soup.find_all('div'):
        try:
            if (stuff.get('class')[0] == 'largernormal') and (stuff.get('style') == 'margin-bottom: 1em;'):
                this_talk['abstract'] = stuff.text[1:-1]
        except:
            pass
    return this_talk

In [4]:
req = requests.get('http://meetings.aps.org/Meeting/MAR22/SessionIndex2')
soup = BeautifulSoup(req.text, 'lxml');
radix = 'http://meetings.aps.org/'
all_sessions = []
all_session_links = []
for link in soup.find_all('a'):
    link_url = str(link.get('href'))
    if '/Meeting/MAR22/Session/' in link_url:
        full_link = "%s%s" % (radix, link_url[1:])
        all_session_links.append(full_link)

In [25]:
all_session_links = sorted(list(set(all_session_links)))

In [38]:
# all_sessions = {}
for idx, session_url in enumerate(all_session_links):
    if session_url in all_sessions:
        continue
    print("%d/%d" % (idx+1, len(all_session_links)), end=' || ')
    parsed_session = parse_session(session_url)
    all_sessions[session_url] = parsed_session
    time.sleep(0.1)

172/962 || Failed...
Failed...
Failed...
Failed...
D18, Failed...
173/962 || D19, 174/962 || D20, 175/962 || D21, 176/962 || D24, 177/962 || D25, 178/962 || D26, 179/962 || D28, 180/962 || D29, 181/962 || D32, 182/962 || D33, 183/962 || D34, 184/962 || D35, 185/962 || D36, 186/962 || D37, 187/962 || D38, 188/962 || D39, 189/962 || D40, 190/962 || D41, 191/962 || D42, 192/962 || D43, 193/962 || D44, 194/962 || D45, 195/962 || D46, 196/962 || D47, 197/962 || D48, 198/962 || D49, 199/962 || D50, 200/962 || D51, 201/962 || D52, 202/962 || D53, 203/962 || D54, 204/962 || D55, 205/962 || D56, 206/962 || D57, 207/962 || D59, 208/962 || D60, 209/962 || D61, 210/962 || D63, 211/962 || D64, 212/962 || D65, 213/962 || D66, 214/962 || D67, 215/962 || D68, 216/962 || D69, 217/962 || D70, 218/962 || D71, Failed...
Failed...
Failed...
Failed...
Failed...
Failed...
Failed...
Failed...
219/962 || D72, 220/962 || E00, 221/962 || E10, 222/962 || E47, 223/962 || E73, 224/962 || F00, 225/962 || F01, 226/96

In [39]:
import pickle

In [40]:
pickle.dump(all_sessions, open('./marchmeet2022.pkl','wb'))

In [49]:
len(all_sessions)

962

In [52]:
the_sessions = {}
good_sessions = []
for session in all_sessions.values():
    if session['id'] not in the_sessions:
        the_sessions[session['id']] = session
        good_sessions.append(session)

In [64]:
for session_id, session in all_sessions.items():
    for talk in session['contents']:
        try:
            authors = [s.lstrip() for s in re.findall(r'content="(.*)" ',talk['html'])[-1].split(';')]
        except:
            authors = []
        talk['authors'] = authors

In [83]:
all_dates = []
for session_id, session in all_sessions.items():
    for talk in session['contents']:
        date = 'TBD'
        try:
            date = re.findall(r'March \d\d',talk['html'])[-1]
        except:
            pass
        talk['date'] = date
        all_dates.append(date)

In [84]:
list(all_sessions.values())[100]['contents'][10]['date']

'March 14'

In [85]:
# fix abstract formatting
for session_id, session in all_sessions.items():
    for talk in session['contents']:
        fixed_abstract = re.sub(r'\.([a-zA-Z])', r'. \1', talk['abstract'])
        talk['abstract'] = fixed_abstract

In [87]:
import pandas as pd

In [ ]:
all_sessions_list = list(all_session.values())

In [92]:
march_meet_df = pd.DataFrame([k['contents'] for k in all_sessions.values()])

In [94]:
all_sessions

NameError: name 'k' is not defined

In [95]:
import pandas as pd

In [96]:
old_meet = pd.read_pickle('/Users/juan/Downloads/march-meeting-2021-df.pkl')

In [164]:
talks = []
bcounter = 0
for session_url, session in all_sessions.items():
    for talk in session['contents']:
        datetime_object = datetime(year=2022,month=3,day=int(talk['date'].split(' ')[-1]))
        talk['datetime'] = datetime_object
        talk['title'] = re.findall(r'Event - (.*)',talk['html'])[0]
        try:
            session_name = list(filter(lambda x: '<h3 style="margin-bottom: 0px;">Session' in x,talk['html'].split('\n')))[0].split('>Session ')[-1].split('</h3>')[0]
            talk['session'] = session_name
        except:
            bcounter += 1
            talk['session'] = ''
        talks.append(talk)


In [161]:
session_name

'Z72: Structural and Phononic Properties of 2D Materials'

In [147]:
old_meet

,html,presenter,abstract,date,authors,title,time_interval,session_id,session,live,on_demand,not_participating,live_links,duration_in_min
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,"The short course focuses on progress, challeng...",2021-03-13,[],DPOLY Short Course: Macromolecular Engineering...,"[8:30AM, 3:15PM]",2A,DPOLY Short Course: Macromolecular Engineering...,False,False,False,,405.0
1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,The purpose of this workshop is to foster the ...,2021-03-14,[],Workshop for the US-Africa Initiative in Elect...,"[8:00AM, 12:00PM]",2B,Workshop for the US-Africa Initiative in Elect...,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,240.0
2,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,Density Functional Theory (DFT) provides a pra...,2021-03-14,[],T1. Density Functional Theory,"[8:30AM, 12:30PM]",3A,T1. Density Functional Theory,False,False,False,,240.0
3,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,"During the course of research, we are often fa...",2021-03-14,[],T2. Active Learning and AI for Computational a...,"[8:30AM, 12:30PM]",3B,T2. Active Learning and AI for Computational a...,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,240.0
4,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,The last few years have seen impressive progre...,2021-03-14,[],T3. Noisy Quantum Devices,"[8:30AM, 12:30PM]",3C,T3. Noisy Quantum Devices,False,False,False,,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10851,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",Hyeonjung Park,We investigate the thermotropic liquid crystal...,2021-03-19,"[Park, Hyeonjung, Han, Moon Jong, Kim, Youngkw...",The Effects of Regioregularity on the Thermotr...,"[2:06PM, 2:18PM]",Y63,Structured Polymers and Their Advanced Morphol...,False,True,False,https://marchmeeting2021.onlineeventpro.freema...,12.0
10852,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",Cornelis Christiaan Bultink,In the race towards creating quantum computers...,2021-03-19,"[Bultink, Cornelis Christiaan, van Oven, Jules...",Qblox & Quantify: fully integrated control har...,"[11:00AM, 12:00PM]",Y66,Exhibitor Workshop: Qblox – Qblox & Quantify: ...,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,60.0
10853,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,DBIO Networking: Microbial Biophysics,2021-03-19,[],DBIO Networking: Microbial Biophysics,"[11:30AM, 1:00PM]",Y80,DBIO Networking: Microbial Biophysics,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,90.0
10854,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,GSNP Networking VI,2021-03-19,[],GSNP Networking VI,"[11:30AM, 1:00PM]",Y81,GSNP Networking VI,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,90.0


In [ ]:
try:
    authors = [s.lstrip() for s in re.findall(r'content="(.*)" ',talk['html'])[-1].split(';')]
except:
    authors = []

In [127]:
atalk = session['contents'][0]

In [129]:
from copy import deepcopy

'Measuring strain fields in moiré superlattices'

In [134]:
this_talk = deepcopy(atalk)
soup = BeautifulSoup(this_talk['html'], 'lxml')
this_talk['html'] = req.text
if '403 Forbidden' in this_talk['html']:
    print("Failed...")
    raise Exception
this_talk['presenter'] = ''
this_talk['abstract'] = ''
for index, thing in enumerate(soup.find_all('h4')):
    print(thing.text)
    if 'Presenter' in thing.text:
        this_talk['presenter'] = (thing.nextSibling.nextSibling.find_all('span')[1].text)
# get the abstract of the presentation
for stuff in soup.find_all('div'):
    try:
        if (stuff.get('class')[0] == 'largernormal') and (stuff.get('style') == 'margin-bottom: 1em;'):
            this_talk['abstract'] = stuff.text[1:-1]
    except:
        pass


Presenter:


Author:

Follow Us
Engage
My APS
Information for
About APS


In [133]:
this_talk['abstract']

"Atomically thin or two-dimensional (2D) materials can be assembled into bespoke heterostructures to produce some extraordinary physical phenomena. One exciting and relatively recent example is the formation of moiré superlattices from azimuthally misoriented (twisted) layers. These moiré superlattices result in flat bands that lead to an array of correlated electronic phases. However, in these systems, complex strain relaxation can also strongly influence the fragile electronic states of the material. Precise characterization of these materials and their properties is therefore critical to the field's understanding of the behavior of these novel moiré materials (and 2D heterostructures in general).This talk will discuss how spontaneous mechanical deformations (atomic reconstruction) and resultant intralayer strain fields at twisted bilayer graphene and twisted transition metal dichalcogenides have been quantitatively imaged using Bragg interferometry, based on four-dimensional scannin

In [125]:
re.findall(r'<title>(.*)</title>',talk['html'])

['Loading...']

In [108]:
from datetime import datetime

In [103]:
old_meet

,html,presenter,abstract,date,authors,title,time_interval,session_id,session,live,on_demand,not_participating,live_links,duration_in_min
0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,"The short course focuses on progress, challeng...",2021-03-13,[],DPOLY Short Course: Macromolecular Engineering...,"[8:30AM, 3:15PM]",2A,DPOLY Short Course: Macromolecular Engineering...,False,False,False,,405.0
1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,The purpose of this workshop is to foster the ...,2021-03-14,[],Workshop for the US-Africa Initiative in Elect...,"[8:00AM, 12:00PM]",2B,Workshop for the US-Africa Initiative in Elect...,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,240.0
2,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,Density Functional Theory (DFT) provides a pra...,2021-03-14,[],T1. Density Functional Theory,"[8:30AM, 12:30PM]",3A,T1. Density Functional Theory,False,False,False,,240.0
3,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,"During the course of research, we are often fa...",2021-03-14,[],T2. Active Learning and AI for Computational a...,"[8:30AM, 12:30PM]",3B,T2. Active Learning and AI for Computational a...,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,240.0
4,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,The last few years have seen impressive progre...,2021-03-14,[],T3. Noisy Quantum Devices,"[8:30AM, 12:30PM]",3C,T3. Noisy Quantum Devices,False,False,False,,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10851,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",Hyeonjung Park,We investigate the thermotropic liquid crystal...,2021-03-19,"[Park, Hyeonjung, Han, Moon Jong, Kim, Youngkw...",The Effects of Regioregularity on the Thermotr...,"[2:06PM, 2:18PM]",Y63,Structured Polymers and Their Advanced Morphol...,False,True,False,https://marchmeeting2021.onlineeventpro.freema...,12.0
10852,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",Cornelis Christiaan Bultink,In the race towards creating quantum computers...,2021-03-19,"[Bultink, Cornelis Christiaan, van Oven, Jules...",Qblox & Quantify: fully integrated control har...,"[11:00AM, 12:00PM]",Y66,Exhibitor Workshop: Qblox – Qblox & Quantify: ...,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,60.0
10853,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,DBIO Networking: Microbial Biophysics,2021-03-19,[],DBIO Networking: Microbial Biophysics,"[11:30AM, 1:00PM]",Y80,DBIO Networking: Microbial Biophysics,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,90.0
10854,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",,GSNP Networking VI,2021-03-19,[],GSNP Networking VI,"[11:30AM, 1:00PM]",Y81,GSNP Networking VI,False,False,False,https://marchmeeting2021.onlineeventpro.freema...,90.0
